In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### Request for Census Tract Level Population data from Census API

In [3]:
stateCode = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
            '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
             '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']

In [4]:
len(stateCode)

51

In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [6]:
def collectAll_county(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs3?get=NAME,'+ variables + '&for=county:*&in=state:*&key=250f58e35283d400e4185b197e3abe9021079afe'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

#### Total population: B01003_001E

In [8]:
year = '2012'
variables = 'B01003_001E'

In [9]:
pop_09 = collectAll_county(year,variables)
pop_09.head()

['NAME', 'B01003_001E', 'state', 'county']


,county_name,B01003_001E,state code,county code
0,"Autauga County, Alabama",55180,01,001
1,"Baldwin County, Alabama",186965,01,003
2,"Barbour County, Alabama",27293,01,005
3,"Bibb County, Alabama",22733,01,007
4,"Blount County, Alabama",57644,01,009


In [10]:
pop_09.shape

(1909, 4)

#### 1.per capita income: B19301_001E

In [11]:
variables = 'B19301_001E'
df1 = collectAll_county(year,variables)

['NAME', 'B19301_001E', 'state', 'county']


In [12]:
df1.shape

(1909, 4)

In [13]:
df1 = df1.rename(columns = {'B19301_001E': 'per_capita_income'})
df1['fips'] = df1['state code'] + df1['county code']
df1.head()

,county_name,per_capita_income,state code,county code,fips
0,"Union County, Georgia",21240,13,291,13291
1,"Upson County, Georgia",17193,13,293,13293
2,"Walker County, Georgia",20316,13,295,13295
3,"Walton County, Georgia",22524,13,297,13297
4,"Ware County, Georgia",17767,13,299,13299


#### 2.Percent of Population Below Poverty Level: (C17002_002E+C17002_003E)/C17002_001E

In [14]:
variables = 'C17002_002E,C17002_003E,C17002_001E'
df2 = collectAll_county(year,variables)

['NAME', 'C17002_002E', 'C17002_003E', 'C17002_001E', 'state', 'county']


In [15]:
df2['C17002_002E'] = df2['C17002_002E'].astype('int')
df2['C17002_003E'] = df2['C17002_003E'].astype('int')
df2['C17002_001E'] = df2['C17002_001E'].astype('int')

In [16]:
df2['percent_below_poverty'] = ((df2['C17002_002E'] + df2['C17002_003E'])/df2['C17002_001E'])*100

In [17]:
df2['fips'] = df2['state code'] + df2['county code']
df2.head()

,county_name,C17002_002E,C17002_003E,C17002_001E,state code,county code,percent_below_poverty,fips
0,"Autauga County, Alabama",3272,3997,54820,01,001,13.259759,01001
1,"Baldwin County, Alabama",9947,14467,184625,01,003,13.223561,01003
2,"Barbour County, Alabama",3155,3091,24435,01,005,25.561694,01005
3,"Bibb County, Alabama",1444,3265,21425,01,007,21.978996,01007
4,"Blount County, Alabama",3655,5265,57124,01,009,15.615153,01009


In [18]:
df2.shape

(1909, 8)

#### 3.Percent of Population 25+ with Less than a 12th Grade Education: (B15002_003E,B15002_004E,B15002_005E,B15002_003E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E)/B15003_001E

In [19]:
def collectAll_county2(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs1?get=NAME,'+ variables + '&for=county:*&in=state:*'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

In [20]:
variables = 'B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E'
df3 = collectAll_county(year,variables)

['NAME', 'B15002_003E', 'B15002_004E', 'B15002_005E', 'B15002_006E', 'B15002_007E', 'B15002_008E', 'B15002_009E', 'B15002_010E', 'B15002_020E', 'B15002_021E', 'B15002_022E', 'B15002_023E', 'B15002_024E', 'B15002_025E', 'B15002_026E', 'B15002_027E', 'B15002_001E', 'state', 'county']


In [21]:
df3.shape

(1909, 20)

In [22]:
def changeInteger(value):
    if value != None:
        return int(value)
    else:
        return np.nan

In [23]:
df3['B15002_003E'] = df3['B15002_003E'].apply(changeInteger)
df3['B15002_004E'] = df3['B15002_004E'].apply(changeInteger)
df3['B15002_005E'] = df3['B15002_005E'].apply(changeInteger)
df3['B15002_006E'] = df3['B15002_006E'].apply(changeInteger)
df3['B15002_007E'] = df3['B15002_007E'].apply(changeInteger)
df3['B15002_008E'] = df3['B15002_008E'].apply(changeInteger)
df3['B15002_009E'] = df3['B15002_009E'].apply(changeInteger)
df3['B15002_010E'] = df3['B15002_010E'].apply(changeInteger)
df3['B15002_020E'] = df3['B15002_020E'].apply(changeInteger)
df3['B15002_021E'] = df3['B15002_021E'].apply(changeInteger)
df3['B15002_022E'] = df3['B15002_022E'].apply(changeInteger)
df3['B15002_023E'] = df3['B15002_023E'].apply(changeInteger)
df3['B15002_024E'] = df3['B15002_024E'].apply(changeInteger)
df3['B15002_025E'] = df3['B15002_025E'].apply(changeInteger)
df3['B15002_026E'] = df3['B15002_026E'].apply(changeInteger)
df3['B15002_027E'] = df3['B15002_027E'].apply(changeInteger)
df3['B15002_001E'] = df3['B15002_001E'].apply(changeInteger)

In [24]:
df3['percent_no_highschool_diploma'] = ((df3['B15002_003E']+df3['B15002_004E']+df3['B15002_005E']+df3['B15002_006E']+df3['B15002_007E']+df3['B15002_008E']+df3['B15002_009E']
                                       +df3['B15002_010E']+df3['B15002_020E']+df3['B15002_021E']+df3['B15002_022E']+df3['B15002_023E']+df3['B15002_024E']+df3['B15002_025E']+df3['B15002_003E']
                                       +df3['B15002_026E']+df3['B15002_027E'])/df3['B15002_001E'])*100

In [25]:
df3['fips'] = df3['state code'] + df3['county code']
df3.head()

,county_name,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,...,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E,state code,county code,percent_no_highschool_diploma,fips
0,"Poinsett County, Arkansas",219.0,132.0,159.0,332.0,353.0,482.0,356.0,241.0,110.0,...,548.0,246.0,385.0,502.0,78.0,16419.0,05,111,28.302576,05111
1,"Polk County, Arkansas",48.0,52.0,46.0,251.0,207.0,252.0,237.0,177.0,123.0,...,219.0,124.0,203.0,199.0,127.0,14356.0,05,113,16.731680,05113
2,"Pope County, Arkansas",295.0,185.0,217.0,693.0,741.0,387.0,830.0,153.0,125.0,...,432.0,585.0,586.0,782.0,108.0,39044.0,05,115,18.151316,05115
3,"Pulaski County, Arkansas",1085.0,525.0,1022.0,1709.0,1423.0,2025.0,2929.0,2038.0,1056.0,...,1826.0,1494.0,3313.0,4223.0,1974.0,257851.0,05,119,11.258440,05119
4,"St. Francis County, Arkansas",250.0,77.0,84.0,394.0,387.0,432.0,374.0,315.0,191.0,...,157.0,300.0,348.0,473.0,100.0,19020.0,05,123,23.175605,05123


#### 4.Percent of Population Living in Mobile Homes: (B25033_006E+B25033_007E+B25033_012E+B25033_013E)/B25033_001E

In [26]:
variables = 'B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E'
df4 = collectAll_county(year,variables)

['NAME', 'B25033_006E', 'B25033_007E', 'B25033_012E', 'B25033_013E', 'B25033_001E', 'state', 'county']


In [27]:
df4.shape

(1909, 8)

In [28]:
df4['B25033_006E'] = df4['B25033_006E'].apply(changeInteger)
df4['B25033_007E'] = df4['B25033_007E'].apply(changeInteger)
df4['B25033_012E'] = df4['B25033_012E'].apply(changeInteger)
df4['B25033_013E'] = df4['B25033_013E'].apply(changeInteger)
df4['B25033_001E'] = df4['B25033_001E'].apply(changeInteger)

In [29]:
df4['percent_mobile_homes'] = ((df4['B25033_006E'] + df4['B25033_007E'] + df4['B25033_012E'] + df4['B25033_013E'])/df4['B25033_001E'])*100

In [30]:
df4['fips'] = df4['state code'] + df4['county code']
df4.head()

,county_name,B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E,state code,county code,percent_mobile_homes,fips
0,"Blaine County, Idaho",1152.0,0.0,587.0,0.0,20876.0,16,013,8.330140,16013
1,"Bonner County, Idaho",5366.0,223.0,2132.0,0.0,40335.0,16,017,19.142184,16017
2,"Bonneville County, Idaho",3412.0,8.0,1961.0,0.0,104591.0,16,019,5.144802,16019
3,"Canyon County, Idaho",9067.0,91.0,3553.0,0.0,188635.0,16,027,6.738410,16027
4,"Cassia County, Idaho",2089.0,0.0,1133.0,0.0,22733.0,16,031,14.173228,16031


#### 5.Percent of Population with No Vehicle Available: (B25044_003E+B25044_010E)/B25044_001E

In [31]:
variables = 'B25044_003E,B25044_010E,B25044_001E'
df5 = collectAll_county(year,variables)

['NAME', 'B25044_003E', 'B25044_010E', 'B25044_001E', 'state', 'county']


In [32]:
df5.shape

(1909, 6)

In [33]:
df5['B25044_003E'] = df5['B25044_003E'].apply(changeInteger)
df5['B25044_010E'] = df5['B25044_010E'].apply(changeInteger)
df5['B25044_001E'] = df5['B25044_001E'].apply(changeInteger)

In [34]:
df5['percent_no_vehicle'] = ((df5['B25044_003E'] + df5['B25044_010E'])/df5['B25044_001E'])*100

In [35]:
df5['fips'] = df5['state code'] + df5['county code']
df5.head()

,county_name,B25044_003E,B25044_010E,B25044_001E,state code,county code,percent_no_vehicle,fips
0,"Jackson County, Florida",374,559,15148,12,063,6.159229,12063
1,"Lake County, Florida",2380,3475,114640,12,069,5.107292,12069
2,"Lee County, Florida",4128,8422,238476,12,071,5.262584,12071
3,"Leon County, Florida",1639,6021,109003,12,073,7.027330,12073
4,"Levy County, Florida",512,673,16180,12,075,7.323857,12075


#### 6.Percent of Population Unemployed:sum(B21005_006E, B21005_007E, B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E)/B21005_001E

In [36]:
variables = 'B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E'
df6 = collectAll_county2(year,variables)

['NAME', 'B21005_006E', 'B21005_007E', 'B21005_011E', 'B21005_012E', 'B21005_017E', 'B21005_022E', 'B21005_028E', 'B21005_029E', 'B21005_033E', 'B21005_034E', 'B21005_001E', 'state', 'county']


In [37]:
df6.shape

(825, 14)

In [38]:
df6['B21005_006E'] = df6['B21005_006E'].apply(changeInteger)
df6['B21005_007E'] = df6['B21005_007E'].apply(changeInteger)
df6['B21005_011E'] = df6['B21005_011E'].apply(changeInteger)
df6['B21005_012E'] = df6['B21005_012E'].apply(changeInteger)
df6['B21005_017E'] = df6['B21005_017E'].apply(changeInteger)
df6['B21005_022E'] = df6['B21005_022E'].apply(changeInteger)
df6['B21005_028E'] = df6['B21005_028E'].apply(changeInteger)
df6['B21005_029E'] = df6['B21005_029E'].apply(changeInteger)
df6['B21005_033E'] = df6['B21005_033E'].apply(changeInteger)
df6['B21005_034E'] = df6['B21005_034E'].apply(changeInteger)
df6['B21005_001E'] = df6['B21005_001E'].apply(changeInteger)

In [39]:
df6['percent_unemployed_CDC'] = ((df6['B21005_006E']+df6['B21005_007E']+df6['B21005_011E']+df6['B21005_012E']+df6['B21005_017E']+df6['B21005_022E']+df6['B21005_028E']+df6['B21005_029E']+df6['B21005_033E']+df6['B21005_034E'])/df6['B21005_001E'])*100

In [40]:
df6['fips'] = df6['state code'] + df6['county code']
df6.head()

,county_name,B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E,state code,county code,percent_unemployed_CDC,fips
0,"Platte County, Missouri",63,75,1197,2316,80,974,13,484,249,3081,58747,29,165,14.523295,29165
1,"St. Francois County, Missouri",52,168,1605,4063,75,756,41,1113,40,3244,41976,29,187,26.579474,29187
2,"St. Louis city, Missouri",38,326,12304,24385,445,7464,174,2971,1426,12342,214565,29,510,28.837415,29510
3,"Christian County, Missouri",0,0,1166,3855,374,666,12,509,412,3020,47601,29,043,21.037373,29043
4,"Jasper County, Missouri",55,65,1659,4127,76,1449,203,573,609,3823,70389,29,097,17.955931,29097


#### 7.Percent of Population Living In Accommodations with Less Than 1 Room Per Person/Crowding: (B25014_005E+B25014_006E+B25014_007E+B25014_011E+B25014_012E+B25014_013E)/B25014_001E

In [41]:
variables = 'B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E'
df7 = collectAll_county(year,variables)

['NAME', 'B25014_005E', 'B25014_006E', 'B25014_007E', 'B25014_011E', 'B25014_012E', 'B25014_013E', 'B25014_001E', 'state', 'county']


In [42]:
df7.shape

(1909, 10)

In [43]:
df7['B25014_005E'] = df7['B25014_005E'].apply(changeInteger)
df7['B25014_006E'] = df7['B25014_006E'].apply(changeInteger)
df7['B25014_007E'] = df7['B25014_007E'].apply(changeInteger)
df7['B25014_011E'] = df7['B25014_011E'].apply(changeInteger)
df7['B25014_012E'] = df7['B25014_012E'].apply(changeInteger)
df7['B25014_013E'] = df7['B25014_013E'].apply(changeInteger)
df7['B25014_001E'] = df7['B25014_001E'].apply(changeInteger)

In [44]:
df7['percent_overcrowding'] = ((df7['B25014_005E']+df7['B25014_006E']+df7['B25014_007E']+df7['B25014_011E']+df7['B25014_012E']+df7['B25014_013E'])/df7['B25014_001E'])*100

In [45]:
df7['fips'] = df7['state code'] + df7['county code']
df7.head()

,county_name,B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E,state code,county code,percent_overcrowding,fips
0,"Jay County, Indiana",74.0,10.0,0.0,53.0,0.0,0.0,8131.0,18,075,1.684910,18075
1,"Jefferson County, Indiana",128.0,0.0,0.0,187.0,0.0,0.0,12664.0,18,077,2.487366,18077
2,"Jennings County, Indiana",106.0,44.0,0.0,46.0,24.0,0.0,10534.0,18,079,2.088475,18079
3,"Johnson County, Indiana",269.0,0.0,0.0,475.0,98.0,80.0,52464.0,18,081,1.757396,18081
4,"Knox County, Indiana",68.0,0.0,0.0,104.0,6.0,2.0,14591.0,18,083,1.233637,18083


#### 8.Percent of Housing Units with 10+ Units in Structure: (B25024_007E+B25024_008E+B25024_009E)/B25024_001E

In [46]:
variables = 'B25024_007E,B25024_008E,B25024_009E,B25024_001E'
df8 = collectAll_county(year,variables)

['NAME', 'B25024_007E', 'B25024_008E', 'B25024_009E', 'B25024_001E', 'state', 'county']


In [47]:
df8.shape

(1909, 7)

In [48]:
df8['B25024_007E'] = df8['B25024_007E'].apply(changeInteger)
df8['B25024_008E'] = df8['B25024_008E'].apply(changeInteger)
df8['B25024_009E'] = df8['B25024_009E'].apply(changeInteger)
df8['B25024_001E'] = df8['B25024_001E'].apply(changeInteger)

In [49]:
df8['percent_multi_unit_housing'] = ((df8['B25024_007E']+df8['B25024_008E']+df8['B25024_009E'])/df8['B25024_001E'])*100

In [50]:
df8['fips'] = df8['state code'] + df8['county code']
df8.head()

,county_name,B25024_007E,B25024_008E,B25024_009E,B25024_001E,state code,county code,percent_multi_unit_housing,fips
0,"Fayette County, Kentucky",9964.0,8153.0,6219.0,135963.0,21,067,17.898987,21067
1,"Floyd County, Kentucky",207.0,43.0,144.0,18186.0,21,071,2.166502,21071
2,"Franklin County, Kentucky",554.0,821.0,403.0,23242.0,21,073,7.649944,21073
3,"Grant County, Kentucky",44.0,25.0,20.0,9957.0,21,081,0.893844,21081
4,"Graves County, Kentucky",194.0,108.0,0.0,16766.0,21,083,1.801264,21083


#### 9.Percent Of Population 65+: (B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_049E)/B01001_001E

In [51]:
variables = 'B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E'
df9 = collectAll_county(year,variables)

['NAME', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E', 'B01001_001E', 'state', 'county']


In [52]:
df9.shape

(1909, 16)

In [53]:
df9['B01001_020E'] = df9['B01001_020E'].apply(changeInteger)
df9['B01001_021E'] = df9['B01001_021E'].apply(changeInteger)
df9['B01001_022E'] = df9['B01001_022E'].apply(changeInteger)
df9['B01001_023E'] = df9['B01001_023E'].apply(changeInteger)
df9['B01001_024E'] = df9['B01001_024E'].apply(changeInteger)
df9['B01001_025E'] = df9['B01001_025E'].apply(changeInteger)
df9['B01001_044E'] = df9['B01001_044E'].apply(changeInteger)
df9['B01001_045E'] = df9['B01001_045E'].apply(changeInteger)
df9['B01001_046E'] = df9['B01001_046E'].apply(changeInteger)
df9['B01001_047E'] = df9['B01001_047E'].apply(changeInteger)
df9['B01001_048E'] = df9['B01001_048E'].apply(changeInteger)
df9['B01001_049E'] = df9['B01001_049E'].apply(changeInteger)
df9['B01001_001E'] = df9['B01001_001E'].apply(changeInteger)

In [54]:
df9['percent_age_65_and_older'] = ((df9['B01001_020E']+df9['B01001_021E']+df9['B01001_022E']+df9['B01001_023E']+df9['B01001_024E']+df9['B01001_025E']+df9['B01001_044E']
                                  +df9['B01001_045E']+df9['B01001_046E']+df9['B01001_047E']+df9['B01001_048E']+df9['B01001_049E'])/df9['B01001_001E'])*100

In [55]:
df9['fips'] = df9['state code'] + df9['county code']
df9.head()

,county_name,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E,state code,county code,percent_age_65_and_older,fips
0,"Escambia County, Florida",3026,3879,4413,3613,2378,1843,3028,4009,6218,4690,3244,3918,300150,12,033,14.745627,12033
1,"Flagler County, Florida",1349,2425,3425,2159,1436,1112,1761,2236,3540,2234,1488,1850,97229,12,035,25.727921,12035
2,"Gadsden County, Florida",237,572,892,539,378,140,535,773,780,670,446,538,47177,12,039,13.777900,12039
3,"Hardee County, Florida",148,425,496,348,190,164,300,220,388,576,94,325,27158,12,049,13.528242,12049
4,"Hendry County, Florida",402,298,675,257,372,366,232,237,690,368,258,307,38387,12,051,11.623727,12051


#### 10.Percent Of Population Who Do Not Speak English: B99163_005E/B99163_001E

In [56]:
variables = 'B99163_005E,B99163_001E'
df10 = collectAll_county(year,variables)

['NAME', 'B99163_005E', 'B99163_001E', 'state', 'county']


In [57]:
df10.shape

(1909, 5)

In [58]:
df10['B99163_005E'] = df10['B99163_005E'].apply(changeInteger)
df10['B99163_001E'] = df10['B99163_001E'].apply(changeInteger)

In [59]:
df10['percent_limited_english_abilities'] = (df10['B99163_005E']/df10['B99163_001E'])*100

In [60]:
df10['fips'] = df10['state code'] + df10['county code']
df10.head()

,county_name,B99163_005E,B99163_001E,state code,county code,percent_limited_english_abilities,fips
0,"Autauga County, Alabama",1345,51643,01,001,2.604419,01001
1,"Baldwin County, Alabama",9903,175445,01,003,5.644504,01003
2,"Barbour County, Alabama",1112,25790,01,005,4.311749,01005
3,"Bibb County, Alabama",379,21565,01,007,1.757477,01007
4,"Blount County, Alabama",3915,54077,01,009,7.239677,01009


#### 11.Percent of Population 17 Years of Age and Under: (B01001_003E+B01001_004E+B01001_005E+B01001_006E+B01001_027E+B01001_028E+B01001_029E+B01001_030E)/B01003_001E

In [61]:
variables = 'B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E'
df11 = collectAll_county(year,variables)

['NAME', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01003_001E', 'state', 'county']


In [62]:
df11.shape

(1909, 12)

In [63]:
df11['B01001_003E'] = df11['B01001_003E'].apply(changeInteger)
df11['B01001_004E'] = df11['B01001_004E'].apply(changeInteger)
df11['B01001_005E'] = df11['B01001_005E'].apply(changeInteger)
df11['B01001_006E'] = df11['B01001_006E'].apply(changeInteger)
df11['B01001_027E'] = df11['B01001_027E'].apply(changeInteger)
df11['B01001_028E'] = df11['B01001_028E'].apply(changeInteger)
df11['B01001_029E'] = df11['B01001_029E'].apply(changeInteger)
df11['B01001_030E'] = df11['B01001_030E'].apply(changeInteger)
df11['B01003_001E'] = df11['B01003_001E'].apply(changeInteger)

In [64]:
df11['percent_age_17_and_younger'] = ((df11['B01001_003E']+df11['B01001_004E']+df11['B01001_005E']+df11['B01001_006E']+df11['B01001_027E']+df11['B01001_028E']+df11['B01001_029E']+df11['B01001_030E'])/df11['B01003_001E'])*100

In [65]:
df11['fips'] = df11['state code'] + df11['county code']
df11.head()

,county_name,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E,state code,county code,percent_age_17_and_younger,fips
0,"Autauga County, Alabama",1860,2220,1890,1267,1677,2204,2082,1253,55180,01,001,26.192461,01001
1,"Baldwin County, Alabama",5765,6074,6076,3770,5755,6127,5703,3333,186965,01,003,22.786618,01003
2,"Barbour County, Alabama",797,812,882,520,706,1134,556,468,27293,01,005,21.525666,01005
3,"Bibb County, Alabama",610,615,735,697,558,964,645,370,22733,01,007,22.847842,01007
4,"Blount County, Alabama",1819,1565,2458,1335,1748,2195,1686,1151,57644,01,009,24.212407,01009


#### 12.Percent Minority: 1-(B03002_003E/B03002_001E)

In [66]:
variables = 'B03002_003E,B03002_001E'
df12 = collectAll_county(year,variables)

['NAME', 'B03002_003E', 'B03002_001E', 'state', 'county']


In [67]:
df12.shape

(1909, 5)

In [68]:
df12['B03002_003E'] = df12['B03002_003E'].apply(changeInteger)
df12['B03002_001E'] = df12['B03002_001E'].apply(changeInteger)

In [69]:
df12['percent_minorities'] = (1 - (df12['B03002_003E']/df12['B03002_001E']))*100

In [70]:
df12['fips'] = df12['state code'] + df12['county code']
df12.head()

,county_name,B03002_003E,B03002_001E,state code,county code,percent_minorities,fips
0,"Calhoun County, Alabama",86436.0,117845.0,01,015,26.652807,01015
1,"Cullman County, Alabama",74350.0,80463.0,01,043,7.597281,01043
2,"Dale County, Alabama",35304.0,50306.0,01,045,29.821492,01045
3,"Houston County, Alabama",69953.0,102538.0,01,069,31.778463,01069
4,"Jefferson County, Alabama",338167.0,659122.0,01,073,48.694324,01073


#### 13.Percent of Population with a Disability: (B18101_004E+B18101_007E+B18101_010E+B18101_013E+B18101_016E+B18101_019E)/B18101_002E + (B18101_023E+B18101_026E+B18101_029E+B18101_032E+B18101_035E+B18101_038E)/B18101_021E

In [71]:
variables = 'B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E'
df13 = collectAll_county2(year,variables)

['NAME', 'B18101_004E', 'B18101_007E', 'B18101_010E', 'B18101_013E', 'B18101_016E', 'B18101_019E', 'B18101_002E', 'B18101_023E', 'B18101_026E', 'B18101_029E', 'B18101_032E', 'B18101_035E', 'B18101_038E', 'B18101_021E', 'state', 'county']


In [72]:
df13.shape

(825, 17)

In [73]:
df13['B18101_004E'] = df13['B18101_004E'].apply(changeInteger)
df13['B18101_007E'] = df13['B18101_007E'].apply(changeInteger)
df13['B18101_010E'] = df13['B18101_010E'].apply(changeInteger)
df13['B18101_013E'] = df13['B18101_013E'].apply(changeInteger)
df13['B18101_016E'] = df13['B18101_016E'].apply(changeInteger)
df13['B18101_002E'] = df13['B18101_002E'].apply(changeInteger)
df13['B18101_023E'] = df13['B18101_023E'].apply(changeInteger)
df13['B18101_026E'] = df13['B18101_026E'].apply(changeInteger)
df13['B18101_029E'] = df13['B18101_029E'].apply(changeInteger)
df13['B18101_032E'] = df13['B18101_032E'].apply(changeInteger)
df13['B18101_035E'] = df13['B18101_035E'].apply(changeInteger)
df13['B18101_038E'] = df13['B18101_038E'].apply(changeInteger)
df13['B18101_021E'] = df13['B18101_021E'].apply(changeInteger)

In [74]:
df13['percent_disabled'] = ((df13['B18101_004E']+df13['B18101_007E']+df13['B18101_010E']+df13['B18101_013E']+df13['B18101_016E'])/df13['B18101_002E'] + (df13['B18101_023E']+df13['B18101_026E']+df13['B18101_029E']+df13['B18101_032E']+df13['B18101_035E']+df13['B18101_038E'])/df13['B18101_021E'])*100

In [75]:
df13['fips'] = df13['state code'] + df13['county code']
df13.head()

,county_name,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E,state code,county code,percent_disabled,fips
0,"Platte County, Missouri",0,475,645,1951,706,1233,45032,0,298,320,1492,541,1504,46255,29,165,17.370182,29165
1,"St. Francois County, Missouri",76,952,308,2779,781,987,29456,0,613,695,2988,531,1369,29913,29,187,37.334804,29187
2,"St. Louis city, Missouri",0,1680,3875,10157,2686,2965,149887,30,725,2980,12848,3556,5024,163410,29,510,27.673271,29510
3,"Christian County, Missouri",0,413,560,1930,806,473,38843,71,108,317,1317,1086,1518,40427,29,043,20.474562,29043
4,"Jasper County, Missouri",0,1091,578,2766,1360,1107,56355,99,678,385,3486,1308,1968,57807,29,097,23.990710,29097


#### 14.Percent of Population Living in Group Quarters: B26001_001E/B01003_001E

In [76]:
variables = 'B26001_001E,B01003_001E'
df14 = collectAll_county(year,variables)

['NAME', 'B26001_001E', 'B01003_001E', 'state', 'county']


In [77]:
df14.shape

(1909, 5)

In [78]:
df14['B26001_001E'] = df14['B26001_001E'].apply(changeInteger)
df14['B01003_001E'] = df14['B01003_001E'].apply(changeInteger)

In [79]:
df14['percent_institutionalized_in_group_quarters'] = (df14['B26001_001E']/df14['B01003_001E'])*100

In [80]:
df14['fips'] = df14['state code'] + df14['county code']
df14.head()

,county_name,B26001_001E,B01003_001E,state code,county code,percent_institutionalized_in_group_quarters,fips
0,"Autauga County, Alabama",423.0,55180,01,001,0.766582,01001
1,"Baldwin County, Alabama",2486.0,186965,01,003,1.329661,01003
2,"Barbour County, Alabama",2860.0,27293,01,005,10.478877,01005
3,"Bibb County, Alabama",1321.0,22733,01,007,5.810936,01007
4,"Blount County, Alabama",578.0,57644,01,009,1.002706,01009


#### 15.Percent of Children Living in Single Parent Households: (B09008_010E+B09008_011E+B09008_012E)/B09008_001E

In [81]:
variables = 'B09008_010E,B09008_011E,B09008_012E,B09008_001E'
df15 = collectAll_county2(year,variables)

['NAME', 'B09008_010E', 'B09008_011E', 'B09008_012E', 'B09008_001E', 'state', 'county']


In [82]:
df15.shape

(825, 7)

In [83]:
df15['B09008_010E'] = df15['B09008_010E'].apply(changeInteger)
df15['B09008_011E'] = df15['B09008_011E'].apply(changeInteger)
df15['B09008_012E'] = df15['B09008_012E'].apply(changeInteger)
df15['B09008_001E'] = df15['B09008_001E'].apply(changeInteger)

In [84]:
df15['percent_single_parent_households_CDC'] = ((df15['B09008_010E'] + df15['B09008_011E']+df15['B09008_012E'])/df15['B09008_001E'])*100

In [85]:
df15['fips'] = df15['state code'] + df15['county code']
df15.head()

,county_name,B09008_010E,B09008_011E,B09008_012E,B09008_001E,state code,county code,percent_single_parent_households_CDC,fips
0,"Oneida County, New York",2254.0,13303.0,191.0,49725.0,36,065,31.670186,36065
1,"Onondaga County, New York",3812.0,24660.0,315.0,103219.0,36,067,27.889245,36067
2,"Ontario County, New York",1019.0,3756.0,20.0,23333.0,36,069,20.550294,36069
3,"Orange County, New York",3117.0,14070.0,130.0,97165.0,36,071,17.822261,36071
4,"Oswego County, New York",1668.0,4659.0,47.0,27081.0,36,075,23.536797,36075


### age 18 to 34
#### (B01001_007E+B01001_008E+B01001_009E+B01001_010E+B01001_011E+B01001_012E+B01001_031E+B01001_032E+B01001_033E+B01001_034E+B01001_035E+B01001_036E)/B01001_001E

In [86]:
variables = 'B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E'
df16 = collectAll_county(year,variables)

['NAME', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_001E', 'state', 'county']


In [87]:
df16['B01001_007E'] = df16['B01001_007E'].apply(changeInteger)
df16['B01001_008E'] = df16['B01001_008E'].apply(changeInteger)
df16['B01001_009E'] = df16['B01001_009E'].apply(changeInteger)
df16['B01001_010E'] = df16['B01001_010E'].apply(changeInteger)
df16['B01001_011E'] = df16['B01001_011E'].apply(changeInteger)
df16['B01001_012E'] = df16['B01001_012E'].apply(changeInteger)
df16['B01001_031E'] = df16['B01001_031E'].apply(changeInteger)
df16['B01001_032E'] = df16['B01001_032E'].apply(changeInteger)
df16['B01001_033E'] = df16['B01001_033E'].apply(changeInteger)
df16['B01001_034E'] = df16['B01001_034E'].apply(changeInteger)
df16['B01001_035E'] = df16['B01001_035E'].apply(changeInteger)
df16['B01001_036E'] = df16['B01001_036E'].apply(changeInteger)
df16['B01001_001E'] = df16['B01001_001E'].apply(changeInteger)

In [88]:
df16['percent_age_18_to_34'] = ((df16['B01001_007E'] + df16['B01001_008E'] +df16['B01001_009E'] +df16['B01001_010E'] +df16['B01001_011E']+df16['B01001_012E']+df16['B01001_031E']+df16['B01001_032E']+df16['B01001_033E']+df16['B01001_034E']+df16['B01001_035E']+df16['B01001_036E'])/df16['B01001_001E'])*100

In [89]:
df16['fips'] = df16['state code'] + df16['county code']
df16.head()

,county_name,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E,state code,county code,percent_age_18_to_34,fips
0,"Escambia County, Florida",7093,3077,3125,7633,11067,9055,5053,2863,2745,7382,10247,8463,300150,12,033,25.921373,12033
1,"Flagler County, Florida",918,810,246,1389,1874,2100,1088,586,476,1006,2131,2898,97229,12,035,15.964373,12035
2,"Gadsden County, Florida",705,459,338,703,1559,1678,432,308,239,718,1626,1296,47177,12,039,21.326070,12039
3,"Hardee County, Florida",759,315,201,459,1048,998,399,46,462,391,623,751,27158,12,049,23.757272,12049
4,"Hendry County, Florida",647,294,524,933,1597,1558,642,216,518,534,953,1128,38387,12,051,24.862584,12051


In [90]:
df16.shape

(1909, 18)

### Age 35 to 44
#### (B01001_013E+B01001_014E+B01001_037E+B01001_038E)/B01001_001E

In [91]:
variables = 'B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E'
df17 = collectAll_county(year,variables)

['NAME', 'B01001_013E', 'B01001_014E', 'B01001_037E', 'B01001_038E', 'B01001_001E', 'state', 'county']


In [92]:
df17['B01001_013E'] = df17['B01001_013E'].apply(changeInteger)
df17['B01001_014E'] = df17['B01001_014E'].apply(changeInteger)
df17['B01001_037E'] = df17['B01001_037E'].apply(changeInteger)
df17['B01001_038E'] = df17['B01001_038E'].apply(changeInteger)
df17['B01001_001E'] = df17['B01001_001E'].apply(changeInteger)

In [93]:
df17['percent_age_35_to_44'] = ((df17['B01001_013E']+df17['B01001_014E'] +df17['B01001_037E']+df17['B01001_038E'])/df17['B01001_001E'])*100

In [94]:
df17['fips'] = df17['state code'] + df17['county code']
df17.head()

,county_name,B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E,state code,county code,percent_age_35_to_44,fips
0,"Escambia County, Florida",8660,8252,8002,8814,300150,12,033,11.237048,12033
1,"Flagler County, Florida",2040,2799,2681,2896,97229,12,035,10.712853,12035
2,"Gadsden County, Florida",1430,1647,1418,1561,47177,12,039,12.836764,12039
3,"Hardee County, Florida",998,959,960,691,27158,12,049,13.285220,12049
4,"Hendry County, Florida",1136,1484,1348,1079,38387,12,051,13.147680,12051


In [95]:
df17.shape

(1909, 10)

### age 45 to 54
#### (B01001_015E+B01001_016E+B01001_039E+B01001_040E)/B01001_001E

In [96]:
variables = 'B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E'
df18 = collectAll_county(year,variables)

['NAME', 'B01001_015E', 'B01001_016E', 'B01001_039E', 'B01001_040E', 'B01001_001E', 'state', 'county']


In [97]:
df18['B01001_015E'] = df18['B01001_015E'].apply(changeInteger)
df18['B01001_016E'] = df18['B01001_016E'].apply(changeInteger)
df18['B01001_039E'] = df18['B01001_039E'].apply(changeInteger)
df18['B01001_040E'] = df18['B01001_040E'].apply(changeInteger)
df18['B01001_001E'] = df18['B01001_001E'].apply(changeInteger)

In [98]:
df18['percent_age_45_to_54'] = ((df18['B01001_015E']+df18['B01001_016E'] +df18['B01001_039E']+df18['B01001_040E'])/df18['B01001_001E'])*100

In [99]:
df18['fips'] = df18['state code'] + df18['county code']
df18.head()

,county_name,B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E,state code,county code,percent_age_45_to_54,fips
0,"Escambia County, Florida",10078,10783,10253,11251,300150,12,033,14.114609,12033
1,"Flagler County, Florida",2815,3431,2805,3524,97229,12,035,12.933384,12035
2,"Gadsden County, Florida",1559,1959,1709,1767,47177,12,039,14.825021,12039
3,"Hardee County, Florida",1087,752,651,698,27158,12,049,11.738714,12049
4,"Hendry County, Florida",1827,879,996,1072,38387,12,051,12.436502,12051


In [100]:
df18.shape

(1909, 10)

### age 55 to 64
#### (B01001_017E+B01001_018E+B01001_019E+B01001_041E+B01001_042E+B01001_043E)/B01001_001E

In [101]:
variables = 'B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E'
df19 = collectAll_county(year,variables)

['NAME', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 'B01001_001E', 'state', 'county']


In [102]:
df19['B01001_017E'] = df19['B01001_017E'].apply(changeInteger)
df19['B01001_018E'] = df19['B01001_018E'].apply(changeInteger)
df19['B01001_019E'] = df19['B01001_019E'].apply(changeInteger)
df19['B01001_041E'] = df19['B01001_041E'].apply(changeInteger)
df19['B01001_042E'] = df19['B01001_042E'].apply(changeInteger)
df19['B01001_043E'] = df19['B01001_043E'].apply(changeInteger)
df19['B01001_001E'] = df19['B01001_001E'].apply(changeInteger)

In [103]:
df19['percent_age_55_to_64'] = ((df19['B01001_017E']+df19['B01001_018E'] +df19['B01001_019E']+df19['B01001_041E']+df19['B01001_042E']+df19['B01001_043E'])/df19['B01001_001E'])*100

In [104]:
df19['fips'] = df19['state code'] + df19['county code']
df19.head()

,county_name,B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E,state code,county code,percent_age_55_to_64,fips
0,"Escambia County, Florida",9127,3182,5656,10775,3720,5591,300150,12,033,12.677328,12033
1,"Flagler County, Florida",2983,1439,2213,3742,2159,2370,97229,12,035,15.330817,12035
2,"Gadsden County, Florida",1761,604,976,1782,510,983,47177,12,039,14.023783,12039
3,"Hardee County, Florida",890,246,298,567,351,452,27158,12,049,10.324766,12049
4,"Hendry County, Florida",1069,390,371,1186,157,645,38387,12,051,9.946075,12051


In [105]:
df19.shape

(1909, 12)

### Combine 19 dataframes together

In [106]:
df = df1[['fips','per_capita_income']].merge(df2[['fips', 'percent_below_poverty']], on = 'fips', how = 'left')

In [107]:
df = df.merge(df3[['fips', 'percent_no_highschool_diploma']], on = 'fips', how = 'left')
df = df.merge(df4[['fips', 'percent_mobile_homes']], on = 'fips', how = 'left')
df = df.merge(df5[['fips', 'percent_no_vehicle']], on = 'fips', how = 'left')
df = df.merge(df7[['fips', 'percent_overcrowding']], on = 'fips', how = 'left')
df = df.merge(df8[['fips', 'percent_multi_unit_housing']], on = 'fips', how = 'left')
df = df.merge(df9[['fips', 'percent_age_65_and_older']], on = 'fips', how = 'left')
df = df.merge(df10[['fips', 'percent_limited_english_abilities']], on = 'fips', how = 'left')
df = df.merge(df12[['fips', 'percent_minorities']], on = 'fips', how = 'left')
df = df.merge(df14[['fips', 'percent_institutionalized_in_group_quarters']], on = 'fips', how = 'left')
df = df.merge(df16[['fips', 'percent_age_18_to_34']], on = 'fips', how = 'left')
df = df.merge(df17[['fips', 'percent_age_35_to_44']], on = 'fips', how = 'left')
df = df.merge(df18[['fips', 'percent_age_45_to_54']], on = 'fips', how = 'left')
df = df.merge(df19[['fips', 'percent_age_55_to_64']], on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,13291,21240,19.038755,15.668938,16.550573,6.688326,NaN,2.509570,27.787138,5.541513,NaN,1.900215,13.891229,9.894224,13.591688,16.947487
1,13293,17193,21.039208,24.885070,13.797291,9.577662,4.133307,1.273624,16.260465,1.544004,NaN,1.942326,18.887442,12.926512,14.746047,14.187907
2,13295,20316,17.531942,21.280834,15.760821,5.766820,2.674467,1.622233,15.315250,2.404949,8.207725,1.901274,19.825486,13.285570,14.830811,13.430026
3,13297,22524,15.923361,17.031771,5.540083,4.975906,2.358088,1.048283,12.468713,5.465659,NaN,0.754457,19.918386,14.373836,14.778348,12.016750
4,13299,17767,25.701757,18.143196,17.397521,10.828121,4.069023,3.035124,15.344823,5.759225,NaN,7.139893,22.755816,12.063958,13.712689,12.570196


In [108]:
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,13291,21240,19.038755,15.668938,16.550573,6.688326,NaN,2.509570,27.787138,5.541513,NaN,1.900215,13.891229,9.894224,13.591688,16.947487
1,13293,17193,21.039208,24.885070,13.797291,9.577662,4.133307,1.273624,16.260465,1.544004,NaN,1.942326,18.887442,12.926512,14.746047,14.187907
2,13295,20316,17.531942,21.280834,15.760821,5.766820,2.674467,1.622233,15.315250,2.404949,8.207725,1.901274,19.825486,13.285570,14.830811,13.430026
3,13297,22524,15.923361,17.031771,5.540083,4.975906,2.358088,1.048283,12.468713,5.465659,NaN,0.754457,19.918386,14.373836,14.778348,12.016750
4,13299,17767,25.701757,18.143196,17.397521,10.828121,4.069023,3.035124,15.344823,5.759225,NaN,7.139893,22.755816,12.063958,13.712689,12.570196


In [109]:
df = df.sort_values('fips')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
1515,01001,24288,13.259759,16.503259,20.691418,5.669717,3.684816,2.994388,12.848858,2.604419,NaN,0.766582,20.416818,14.534252,15.166727,10.840884
1516,01003,26854,13.223561,11.200963,12.849701,3.388881,1.517552,18.532413,17.147594,5.644504,NaN,1.329661,19.069879,12.691145,14.513946,13.790816
1517,01005,16337,25.561694,26.584544,23.726108,9.427464,NaN,0.987842,14.912249,4.311749,NaN,10.478877,22.045946,13.955959,14.446928,13.113252
1518,01007,15270,21.978996,28.814993,32.850738,3.264387,NaN,NaN,13.887300,1.757477,NaN,5.810936,22.641094,13.803721,15.466502,11.353539
1519,01009,20304,15.615153,24.195808,21.557495,3.690072,1.274066,1.390343,15.273055,7.239677,NaN,1.002706,19.986469,13.369995,14.079523,13.078551


In [110]:
df.to_csv('SDoH_3years/Social_Economic_3years_2012.csv')